In [1]:
import sys; sys.path.append('../util')
from load_yelp_data import load_yelp_dataframe, restaurants_and_bars_in, train_test_split_reviews
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.metrics
import scipy.optimize
%matplotlib inline

from baseline_calculator import *

In [2]:
businesses = load_yelp_dataframe('businesses')
reviews = load_yelp_dataframe('reviews')
users = load_yelp_dataframe('users')
reviews['date']=pd.to_datetime(reviews.year*10000+reviews.month*100+reviews.day,format='%Y%m%d')
phoenix_restaurants, phoenix_reviews, phoenix_users = restaurants_and_bars_in('Phoenix', businesses, reviews, users)

In [3]:
reviews_train, reviews_test = train_test_split_reviews(phoenix_reviews)

In [4]:
import surprise

def dump_to_file(revs, fname):
    with open(fname, 'w') as f:
        for r in revs.itertuples():
            f.write('{}\t{}\t{}\t{}\n'.format(r.user_id, r.business_id, r.stars, r.date))   
dump_to_file(reviews_train, 'phoenix_reviews.data')
dump_to_file(reviews_test, 'phoenix_reviews.test')

reader = surprise.Reader(line_format='user item rating timestamp', sep='\t')
data = surprise.Dataset.load_from_folds([('phoenix_reviews.data', 'phoenix_reviews.test')], reader=reader)

svd_pp = surprise.prediction_algorithms.matrix_factorization.SVDpp()
surprise.evaluate(svd_pp, data)

In [5]:
import surprise

In [6]:
reader = surprise.Reader(line_format='user item rating timestamp', sep='\t')

In [7]:
data = surprise.Dataset.load_from_folds([('phoenix_reviews.data', 'phoenix_reviews.test')], reader=reader)

In [8]:
baseline = surprise.BaselineOnly()

In [9]:
surprise.evaluate(baseline, data)

Evaluating RMSE, MAE of algorithm BaselineOnly.

------------
Fold 1
Estimating biases using als...
RMSE: 1.2328
MAE:  0.9943
------------
------------
Mean RMSE: 1.2328
Mean MAE : 0.9943
------------
------------


CaseInsensitiveDefaultDict(list,
                           {u'mae': [0.99428425511368945],
                            u'rmse': [1.2328176091613863]})

In [10]:
svd_pp = surprise.prediction_algorithms.matrix_factorization.SVDpp()
surprise.evaluate(svd_pp, data)

Evaluating RMSE, MAE of algorithm SVDpp.

------------
Fold 1
RMSE: 1.2344
MAE:  0.9943
------------
------------
Mean RMSE: 1.2344
Mean MAE : 0.9943
------------
------------


CaseInsensitiveDefaultDict(list,
                           {u'mae': [0.99434851054475049],
                            u'rmse': [1.2344357910538006]})

In [11]:
from baseline_calculator import DecoupledRegularizedBaselineCalculator

bc = DecoupledRegularizedBaselineCalculator()
bc.fit(reviews_train, busi_reg_strength=2.5, user_reg_strength=5)
bc.transform(reviews_train, key='baseline_stars')
bc.transform(reviews_test, key='baseline_stars')
bc.baseline_rmse(reviews_test)

1.2247996442394655

In [12]:
def dump_baselines_to_file(revs, fname):
    with open(fname, 'w') as f:
        for r in revs.itertuples():
            f.write('{}\t{}\t{}\t{}\n'.format(r.user_id, r.business_id, r.baseline_stars, r.date))
            
dump_baselines_to_file(reviews_train, 'phoenix_baselines.data')
dump_baselines_to_file(reviews_test, 'phoenix_baselines.test')

In [13]:
reader2 = surprise.Reader(line_format='user item rating timestamp', sep='\t')
baselines = surprise.Dataset.load_from_folds([('phoenix_baselines.data', 'phoenix_baselines.test')], reader=reader2)

In [22]:
fold = [f for f in baselines.folds()][0]

In [28]:
bl_train, bl_test = fold

In [30]:
svd_pp2 = surprise.prediction_algorithms.matrix_factorization.SVDpp()
svd_pp2.train(bl_train)

In [31]:
predictions = svd_pp2.test(bl_test)

In [32]:
predictions[0]

Prediction(uid=u'658647', iid=u'2377', r0=1.25479164701, est=1, details={u'was_impossible': False})

In [37]:
pred_bls = np.array([p.est for p in predictions])

In [38]:
from sklearn import metrics
np.sqrt(metrics.mean_squared_error(reviews_test.stars, bc.inverse_transform(reviews_test, pred_bls)))

1.5233207548656456

In [40]:
len(reviews_train.business_id.unique())

10154

In [41]:
len(reviews_train.user_id.unique())

140465